In [1]:
import tensorflow as tf
import pandas

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from PIL import Image
from pathlib import Path
from skimage import io
from skimage import transform
from sklearn.utils import class_weight

In [3]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt

In [4]:
import tensorflow_datasets as tfds

In [5]:
path = os.getcwd()
path = os.path.join(path, "gtsrb-german-traffic-sign")

In [6]:
batch_size = 64
epochs = 30
INIT_LR = 1e-3
IMG_HEIGHT = 32
IMG_WIDTH = 32
numClasses = 43

In [7]:
train_image_generator = ImageDataGenerator(rescale=1./255, 
                                           rotation_range=10, 
                                           zoom_range=0.15,
                                           width_shift_range=0.1,
                                           height_shift_range=0.1,
                                           shear_range=0.15,
                                           horizontal_flip=False,
                                           vertical_flip=False,
                                           fill_mode="nearest")

test_image_generator = ImageDataGenerator(rescale=1./255)

In [8]:
train_csv = os.path.join(path, "Train.csv")
train_df = pandas.read_csv(train_csv, dtype=str, usecols = ["Path","ClassId"])

In [9]:
train_data_gen = train_image_generator.flow_from_dataframe(
        dataframe=train_df,
        directory=path,
        shuffle=True,
        x_col="Path",
        y_col="ClassId",
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

Found 39209 validated image filenames belonging to 43 classes.


In [10]:
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_data_gen.classes), 
                train_data_gen.classes)

In [11]:
train_data, train_targets = next(train_data_gen)

In [12]:
test_csv = os.path.join(path, "Test.csv")
valid_df = pandas.read_csv(test_csv, dtype=str, usecols = ["Path","ClassId"])

In [13]:
val_data_gen = test_image_generator.flow_from_dataframe(
        dataframe=valid_df,
        directory=path,
        x_col="Path",
        y_col="ClassId",
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

Found 12630 validated image filenames belonging to 43 classes.


In [14]:
valid_data, valid_targets = next(val_data_gen)

In [15]:
model = Sequential([
    Conv2D(8, 5, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(16, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(16, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    Conv2D(32, 3, padding='same', activation='relu'),
    BatchNormalization(axis=-1),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(numClasses, activation='softmax')
])

In [16]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / (epochs * 0.5))

In [17]:
model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [18]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=500,
    epochs=epochs,
    validation_data=val_data_gen,
    class_weight=class_weights,
    verbose = 1
)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 500 steps, validate for 198 steps
Epoch 1/30
500/500 [==============================] - 53s 105ms/step - loss: 3.2335 - accuracy: 0.1840 - val_loss: 2.0070 - val_accuracy: 0.3850
Epoch 2/30
500/500 [==============================] - 58s 115ms/step - loss: 1.8509 - accuracy: 0.4249 - val_loss: 1.2496 - val_accuracy: 0.5867
Epoch 3/30
500/500 [==============================] - 57s 115ms/step - loss: 1.3105 - accuracy: 0.5734 - val_loss: 0.9829 - val_accuracy: 0.6761
Epoch 4/30
500/500 [==============================] - 57s 114ms/step - loss: 0.9897 - accuracy: 0.6746 - val_loss: 0.6279 - val_accuracy: 0.7946
Epoch 5/30
500/500 [==============================] - 58s 116ms/step - loss: 0.7694 - accuracy: 0.7485 - val_loss: 0.5013 - val_accuracy: 0.8378
Epoch 6/30
500/500 [==============================] - 57s 114ms/step - loss: 0.5940 - accuracy: 0.8088 - 

In [19]:
model.save("TrafficSignRec")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: TrafficSignRec\assets
